#### Setup

In [1]:
import torch
import torch.nn as nn
from transformers import RobertaModel, RobertaTokenizer
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from torch.optim.lr_scheduler import ExponentialLR
from sklearn.model_selection import train_test_split

import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

----------------------------------------------------------
To fix the error `Torch compile: libcuda.so cannot found` raised by
```python
torch.compile(robertaModel, backend="inductor")
```
----------------------------------------------------------

In [11]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link



In [2]:
# Check for GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


#### Dataset

In [3]:
train_csv_url = "/content/drive/MyDrive/#Semester07/FYP/FakeNews/fake-news/train.csv"
train_data = pd.read_csv(train_csv_url)
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
test_csv_url = "/content/drive/MyDrive/#Semester07/FYP/FakeNews/fake-news/test.csv"
test_data = pd.read_csv(test_csv_url)
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [5]:
print(f"train dataset shape {train_data.shape}")
print(f"# of missing values {train_data.isna().sum()}")
print(f"label summary\n{train_data['label'].value_counts()}")

train dataset shape (20800, 5)
# of missing values id           0
title      558
author    1957
text        39
label        0
dtype: int64
label summary
1    10413
0    10387
Name: label, dtype: int64


In [6]:
filtered_train_data = train_data.copy()

# Remove missing values in "text" column
print(f"missing value count {filtered_train_data['text'].isna().sum()}")
filtered_train_data.dropna(subset=['text'], inplace=True)

# Check for empty strings and drop rows with empty "text" values
filtered_train_data['text'] = filtered_train_data['text'].str.strip() # Strip whitespace from the "text" column
print(f"empty string count {filtered_train_data[filtered_train_data['text'] == ''].shape[0]}")
filtered_train_data = filtered_train_data[filtered_train_data['text'] != '']
print(f"filtered_train_data dataset shape {filtered_train_data.shape}")

missing value count 39
empty string count 77
filtered_train_data dataset shape (20684, 5)


In [7]:
x_train, x_temp, y_train, y_temp = train_test_split(filtered_train_data['text'], filtered_train_data['label'], test_size=0.2, stratify=filtered_train_data['label'], random_state=2023)

In [8]:
x_test, x_valid, y_test, y_valid = train_test_split(x_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=2023)

In [9]:
print(f"y_train\n {y_train.value_counts()}")
print(f"y_test\n {y_test.value_counts()}")
print(f"y_valid\n {y_valid.value_counts()}")

y_train
 0    8309
1    8238
Name: label, dtype: int64
y_test
 0    1039
1    1029
Name: label, dtype: int64
y_valid
 0    1039
1    1030
Name: label, dtype: int64


#### Implementation

In [3]:
class FakeBERT(nn.Module):
    def __init__(
        self,
        device,
        roberta_model_path='roberta-base',
        num_classes=1,
        inductor=True
        ):
        super(FakeBERT, self).__init__()

        # Load pre-trained RoBERTa model
        self.roberta = RobertaModel.from_pretrained(roberta_model_path).to(device=device)
        if (inductor):
          self.roberta = torch.compile(self.roberta, backend="inductor")
        self.tokenizer = RobertaTokenizer.from_pretrained(roberta_model_path)

        # CNN
        self.conv1d_p1 = nn.Conv1d(in_channels=768, out_channels=128, kernel_size=5).to(device=device)
        self.conv1d_p2 = nn.Conv1d(in_channels=768, out_channels=128, kernel_size=4).to(device=device)
        self.conv1d_p3 = nn.Conv1d(in_channels=768, out_channels=128, kernel_size=3).to(device=device)
        self.conv1d_s1 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=5).to(device=device)
        self.conv1d_s2 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=5).to(device=device)

        # Pooling
        self.max_pool_p1 = nn.MaxPool1d(kernel_size=5).to(device=device)
        self.max_pool_p2 = nn.MaxPool1d(kernel_size=5).to(device=device)
        self.max_pool_p3 = nn.MaxPool1d(kernel_size=5).to(device=device)
        self.max_pool_s1 = nn.MaxPool1d(kernel_size=5).to(device=device)
        self.max_pool_s2 = nn.MaxPool1d(kernel_size=10).to(device=device)

        # Fully connected layers
        self.linear1 = nn.Linear(640, 128).to(device=device)
        self.linear2 = nn.Linear(128, num_classes).to(device=device)
        self.sigmoid = nn.Sigmoid().to(device=device)

    def forward(self, x):
        # Tokenize and encode the sentences
        tokenized_sentences = self.tokenizer(x, truncation=True, padding='max_length', return_tensors='pt').to(device=device)
        # print('tokenized_sentences', tokenized_sentences.shape)

        # Forward pass to get embeddings
        with torch.no_grad():
            # Get RoBERTa embeddings
            model_output = self.roberta(**tokenized_sentences)

        # Extract embeddings from the output
        embeddings = model_output.last_hidden_state
        # print('embeddings', embeddings.shape)

        output_p1 = self.max_pool_p1(F.relu(self.conv1d_p1(embeddings.permute(0, 2, 1))))
        output_p2 = self.max_pool_p2(F.relu(self.conv1d_p2(embeddings.permute(0, 2, 1))))
        output_p3 = self.max_pool_p3(F.relu(self.conv1d_p3(embeddings.permute(0, 2, 1))))
        output_s = torch.cat((output_p1, output_p2, output_p3), dim=2)
        output_s1 = F.relu(self.conv1d_s1(output_s))
        output_s1 = self.max_pool_s1(output_s1)
        output_s2 = F.relu(self.conv1d_s2(output_s1))
        output_s2 = self.max_pool_s2(output_s2)
        output_s2 = output_s2.permute(0, 2, 1)
        output_f = output_s2.reshape(output_s2.size(0), -1)
        output_l1 = torch.relu(self.linear1(output_f))
        output_l2 = self.linear2(output_l1)
        output = self.sigmoid(output_l2)

        return output

In [13]:
sentences = ["I love this product!", "It's terrible.", "Awesome experience.", "Worst ever.", "Great job!", "Awful.", "Excellent service.", "Hate it.", "Fantastic!", "Disappointing."]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]

# x_train, x_valid, y_train, y_valid = train_test_split(sentences, labels, test_size=0.2, stratify=labels, random_state=2023)

# Create DataLoader for training and validation sets
batch_size = 128

train_dataset = list(zip(x_train, y_train))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = list(zip(x_valid, y_valid))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model
fakebert = FakeBERT(
    device,
    inductor=False
)

# Loss function and optimizer
criterion = nn.BCELoss().to(device=device)  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(fakebert.parameters(), lr=0.001)

d_r = 10**-10
scheduler = ExponentialLR(optimizer, gamma=(1.0 - d_r))

# Define early stopping parameters
patience = 5
best_validation_accuracy = 0
no_improvement_counter = 0

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    fakebert.train()
    total_correct_train = 0
    total_samples_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = fakebert(inputs)

        # Compute training accuracy
        predicted_labels_train = (outputs > 0.5).int()
        total_correct_train += (predicted_labels_train.view(-1) == labels.to(device=device)).sum().item()

        # Accumulate the sum of batch sizes
        total_samples_train += len(labels)

        loss = criterion(outputs, torch.as_tensor(labels, dtype=torch.float32).unsqueeze(1).to(device=device))
        loss.backward()
        optimizer.step()

    # Compute training accuracy
    train_accuracy = total_correct_train / total_samples_train

    # Validation
    val_loss = 0.0
    total_correct_val = 0
    total_samples_val = 0
    num_iterations = (len(y_valid) + batch_size - 1) // batch_size

    fakebert.eval()
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = fakebert(val_inputs)

            # Compute validation accuracy
            predicted_labels_val = (val_outputs > 0.5).int()
            total_correct_val += (predicted_labels_val.view(-1) == val_labels.to(device=device)).sum().item()

            # Accumulate the sum of batch sizes
            total_samples_val += len(val_labels)

            val_batch_targets = torch.as_tensor(val_labels, dtype=torch.float32).unsqueeze(1).to(device=device)
            ce = criterion(val_outputs, val_batch_targets).item()
            val_loss += ce

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = total_correct_val / total_samples_val

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_val_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')

        # Check for early stopping
        if val_accuracy > best_validation_accuracy:
            best_validation_accuracy = val_accuracy
            no_improvement_counter = 0
            # Save the trained best fakebert if needed
            torch.save(fakebert.state_dict(), '/content/drive/Shareddrives/test/FYP/fake-news/fakebert.pth')
        else:
            no_improvement_counter += 1

        # If no improvement for 'patience' consecutive epochs, stop training
        if no_improvement_counter >= patience:
            print("Early stopping triggered. Training stopped.")
            break

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10, Loss: 0.0057, Training Accuracy: 0.9527, Validation Accuracy: 0.9976
Epoch 2/10, Loss: 0.0120, Training Accuracy: 0.9966, Validation Accuracy: 0.9947
Epoch 3/10, Loss: 0.0037, Training Accuracy: 0.9981, Validation Accuracy: 0.9995
Epoch 4/10, Loss: 0.0025, Training Accuracy: 0.9988, Validation Accuracy: 0.9990
Epoch 5/10, Loss: 0.0031, Training Accuracy: 0.9996, Validation Accuracy: 0.9990
Epoch 6/10, Loss: 0.0023, Training Accuracy: 0.9991, Validation Accuracy: 0.9990
Epoch 7/10, Loss: 0.0019, Training Accuracy: 0.9998, Validation Accuracy: 0.9990
Epoch 8/10, Loss: 0.0018, Training Accuracy: 0.9997, Validation Accuracy: 0.9995
Early stopping triggered. Training stopped.


#### Twitter US Airline Sentiment

In [5]:
file_path = '/content/drive/Shareddrives/test/Sentiment/Tweets.csv'

df = pd.read_csv(file_path, usecols=['airline_sentiment', 'text'])
df.dropna(subset=['text'], inplace=True)
df.dropna(subset=['airline_sentiment'], inplace=True)

df['airline_sentiment'] = df['airline_sentiment'].map({'neutral': 2, 'positive': 1, 'negative': 0})

valid_sentiments = [0, 1]
df_valid = df[df['airline_sentiment'].isin(valid_sentiments)]

In [6]:
df_valid['airline_sentiment'].value_counts()

0    9178
1    2363
Name: airline_sentiment, dtype: int64

In [13]:
# Split the data into training and validation sets
x_train, x_temp, y_train, y_temp = train_test_split(df_valid['text'], df_valid['airline_sentiment'], test_size=0.2, stratify=df_valid['airline_sentiment'], random_state=2023)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=2023)

batch_size = 128
criterion = nn.BCELoss().to(device=device)

In [10]:
# Create DataLoader for training and validation sets
batch_size = 128

train_dataset = list(zip(x_train, y_train))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = list(zip(x_val, y_val))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model
fakebert = FakeBERT(
    device,
    inductor=True
)

# Loss function and optimizer
criterion = nn.BCELoss().to(device=device)  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(fakebert.parameters(), lr=0.001)

d_r = 10**-10
scheduler = ExponentialLR(optimizer, gamma=(1.0 - d_r))

# Define early stopping parameters
patience = 5
best_validation_accuracy = 0
no_improvement_counter = 0

# Training loop
num_epochs = 20

for epoch in range(num_epochs):
    fakebert.train()
    total_correct_train = 0
    total_samples_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = fakebert(inputs)

        # Compute training accuracy
        predicted_labels_train = (outputs > 0.5).int()
        total_correct_train += (predicted_labels_train.view(-1) == labels.to(device=device)).sum().item()

        # Accumulate the sum of batch sizes
        total_samples_train += len(labels)

        loss = criterion(outputs, torch.as_tensor(labels, dtype=torch.float32).unsqueeze(1).to(device=device))
        loss.backward()
        optimizer.step()

    # Compute training accuracy
    train_accuracy = total_correct_train / total_samples_train

    # Validation
    val_loss = 0.0
    total_correct_val = 0
    total_samples_val = 0
    num_iterations = (len(y_val) + batch_size - 1) // batch_size

    fakebert.eval()
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = fakebert(val_inputs)

            # Compute validation accuracy
            predicted_labels_val = (val_outputs > 0.5).int()
            total_correct_val += (predicted_labels_val.view(-1) == val_labels.to(device=device)).sum().item()

            # Accumulate the sum of batch sizes
            total_samples_val += len(val_labels)

            val_batch_targets = torch.as_tensor(val_labels, dtype=torch.float32).unsqueeze(1).to(device=device)
            ce = criterion(val_outputs, val_batch_targets).item()
            val_loss += ce

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = total_correct_val / total_samples_val

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_val_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')

        # Check for early stopping
        if val_accuracy > best_validation_accuracy:
            best_validation_accuracy = val_accuracy
            no_improvement_counter = 0
            # Save the trained best fakebert if needed
            torch.save(fakebert.state_dict(), '/content/drive/Shareddrives/test/FYP/fake-news/fakebert-twitterus.pth')
        else:
            no_improvement_counter += 1

        # If no improvement for 'patience' consecutive epochs, stop training
        if no_improvement_counter >= patience:
            print("Early stopping triggered. Training stopped.")
            break

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20, Loss: 0.1972, Training Accuracy: 0.8450, Validation Accuracy: 0.9273
Epoch 2/20, Loss: 0.1721, Training Accuracy: 0.9334, Validation Accuracy: 0.9385
Epoch 3/20, Loss: 0.1931, Training Accuracy: 0.9424, Validation Accuracy: 0.9255
Epoch 4/20, Loss: 0.1475, Training Accuracy: 0.9472, Validation Accuracy: 0.9481
Epoch 5/20, Loss: 0.1569, Training Accuracy: 0.9541, Validation Accuracy: 0.9489
Epoch 6/20, Loss: 0.1377, Training Accuracy: 0.9586, Validation Accuracy: 0.9541
Epoch 7/20, Loss: 0.1501, Training Accuracy: 0.9641, Validation Accuracy: 0.9506
Epoch 8/20, Loss: 0.1713, Training Accuracy: 0.9702, Validation Accuracy: 0.9307
Epoch 9/20, Loss: 0.1580, Training Accuracy: 0.9747, Validation Accuracy: 0.9481
Epoch 10/20, Loss: 0.1831, Training Accuracy: 0.9816, Validation Accuracy: 0.9411
Epoch 11/20, Loss: 0.2096, Training Accuracy: 0.9884, Validation Accuracy: 0.9446
Early stopping triggered. Training stopped.


In [14]:
test_dataset = list(zip(x_test, y_test))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create an instance of the model
fakebertTwitterUS = FakeBERT(
    device,
    inductor=True
)

# Load the saved model state dictionary
fakebertTwitterUS.load_state_dict(torch.load('/content/drive/Shareddrives/test/FYP/fake-news/fakebert-twitterus.pth'))

# Testing
test_loss = 0.0
total_correct_test = 0
total_samples_test = 0
num_iterations = (len(y_test) + batch_size - 1) // batch_size

fakebertTwitterUS.eval()
with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = fakebertTwitterUS(test_inputs)

        # Compute validation accuracy
        predicted_labels_test = (test_outputs > 0.5).int()
        total_correct_test += (predicted_labels_test.view(-1) == test_labels.to(device=device)).sum().item()

        # Accumulate the sum of batch sizes
        total_samples_test += len(test_labels)

        test_batch_targets = torch.as_tensor(test_labels, dtype=torch.float32).unsqueeze(1).to(device=device)
        ce = criterion(test_outputs, test_batch_targets).item()
        test_loss += ce

    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = total_correct_test / total_samples_test

    print(f'Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.1103, Test Accuracy: 0.9558


#### IMDB Sentiment

In [15]:
file_path = '/content/drive/Shareddrives/test/Sentiment/IMDB Dataset.csv'

df = pd.read_csv(file_path, usecols=['sentiment', 'review'])
df.dropna(subset=['review'], inplace=True)
df.dropna(subset=['sentiment'], inplace=True)

df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

valid_sentiments = [0, 1]
df_valid = df[df['sentiment'].isin(valid_sentiments)]
df_valid['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [16]:
x_train, x_temp, y_train, y_temp = train_test_split(df_valid['review'], df_valid['sentiment'], test_size=0.2, stratify=df_valid['sentiment'], random_state=2023)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=2023)

print(f"y_train\n {y_train.value_counts()}")
print(f"y_test\n {y_test.value_counts()}")
print(f"y_val\n {y_val.value_counts()}")

y_train
 1    20000
0    20000
Name: sentiment, dtype: int64
y_test
 1    2500
0    2500
Name: sentiment, dtype: int64
y_val
 1    2500
0    2500
Name: sentiment, dtype: int64


In [ ]:
# Create DataLoader for training and validation sets
batch_size = 128

train_dataset = list(zip(x_train, y_train))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = list(zip(x_val, y_val))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model
fakebert = FakeBERT(
    device,
    inductor=True
)

# Loss function and optimizer
criterion = nn.BCELoss().to(device=device)  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(fakebert.parameters(), lr=0.001)

d_r = 10**-10
scheduler = ExponentialLR(optimizer, gamma=(1.0 - d_r))

# Define early stopping parameters
patience = 5
best_validation_accuracy = 0
no_improvement_counter = 0

# Training loop
num_epochs = 20

for epoch in range(num_epochs):
    fakebert.train()
    total_correct_train = 0
    total_samples_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = fakebert(inputs)

        # Compute training accuracy
        predicted_labels_train = (outputs > 0.5).int()
        total_correct_train += (predicted_labels_train.view(-1) == labels.to(device=device)).sum().item()

        # Accumulate the sum of batch sizes
        total_samples_train += len(labels)

        loss = criterion(outputs, torch.as_tensor(labels, dtype=torch.float32).unsqueeze(1).to(device=device))
        loss.backward()
        optimizer.step()

    # Compute training accuracy
    train_accuracy = total_correct_train / total_samples_train

    # Validation
    val_loss = 0.0
    total_correct_val = 0
    total_samples_val = 0
    num_iterations = (len(y_val) + batch_size - 1) // batch_size

    fakebert.eval()
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = fakebert(val_inputs)

            # Compute validation accuracy
            predicted_labels_val = (val_outputs > 0.5).int()
            total_correct_val += (predicted_labels_val.view(-1) == val_labels.to(device=device)).sum().item()

            # Accumulate the sum of batch sizes
            total_samples_val += len(val_labels)

            val_batch_targets = torch.as_tensor(val_labels, dtype=torch.float32).unsqueeze(1).to(device=device)
            ce = criterion(val_outputs, val_batch_targets).item()
            val_loss += ce

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = total_correct_val / total_samples_val

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_val_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')

        # Check for early stopping
        if val_accuracy > best_validation_accuracy:
            best_validation_accuracy = val_accuracy
            no_improvement_counter = 0
            # Save the trained best fakebert if needed
            torch.save(fakebert.state_dict(), '/content/drive/Shareddrives/test/FYP/fake-news/fakebert-imdb.pth')
        else:
            no_improvement_counter += 1

        # If no improvement for 'patience' consecutive epochs, stop training
        if no_improvement_counter >= patience:
            print("Early stopping triggered. Training stopped.")
            break

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20, Loss: 0.1729, Training Accuracy: 0.8937, Validation Accuracy: 0.9370
Epoch 2/20, Loss: 0.1676, Training Accuracy: 0.9330, Validation Accuracy: 0.9378
Epoch 3/20, Loss: 0.1912, Training Accuracy: 0.9433, Validation Accuracy: 0.9290
Epoch 4/20, Loss: 0.1697, Training Accuracy: 0.9507, Validation Accuracy: 0.9396
Epoch 5/20, Loss: 0.1674, Training Accuracy: 0.9584, Validation Accuracy: 0.9382
Epoch 6/20, Loss: 0.1757, Training Accuracy: 0.9668, Validation Accuracy: 0.9412
Epoch 7/20, Loss: 0.1954, Training Accuracy: 0.9747, Validation Accuracy: 0.9412
Epoch 8/20, Loss: 0.2235, Training Accuracy: 0.9796, Validation Accuracy: 0.9378


In [ ]:
test_dataset = list(zip(x_test, y_test))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create an instance of the model
fakebertIMDB = FakeBERT(
    device,
    inductor=True
)

# Load the saved model state dictionary
fakebertIMDB.load_state_dict(torch.load('/content/drive/Shareddrives/test/FYP/fake-news/fakebert-imdb.pth'))

# Testing
test_loss = 0.0
total_correct_test = 0
total_samples_test = 0
num_iterations = (len(y_test) + batch_size - 1) // batch_size

fakebertIMDB.eval()
with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = fakebertIMDB(test_inputs)

        # Compute validation accuracy
        predicted_labels_test = (test_outputs > 0.5).int()
        total_correct_test += (predicted_labels_test.view(-1) == test_labels.to(device=device)).sum().item()

        # Accumulate the sum of batch sizes
        total_samples_test += len(test_labels)

        test_batch_targets = torch.as_tensor(test_labels, dtype=torch.float32).unsqueeze(1).to(device=device)
        ce = criterion(test_outputs, test_batch_targets).item()
        test_loss += ce

    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = total_correct_test / total_samples_test

    print(f'Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')